# Teaching a Turkish language model new tricks

Instead of creating a new Machine Learning (ML) model for every new task, we can leverage the concept of *Transfer Learning*. 

In particular, we can use generic language models and teach it new tasks by fine-tuning them using corresponding datasets. In this notebook we will use a Turkish language model created by the MDZ Digital Library team (dbmdz) at the Bavarian State Library (https://github.com/stefan-it/turkish-bert). We will use the Hugging Face Model Hub to download the model (https://huggingface.co/dbmdz/bert-base-turkish-uncased) and then fine-tune it to 2 very different tasks:

- Sentiment Analysis
- Question Answering

## SageMaker Setup

In [ ]:
!pip install transformers -q -U

In [ ]:
!pip install datasets -q -U

In [ ]:
!mkdir data

In [61]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session_bucket}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20211005T160629
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


#### Define model name

In [1]:
model_name = 'dbmdz/bert-base-turkish-uncased'

## Sentiment Analysis

### Downloading dataset and splitting into test and training sets

We will downlaod the data directly from Huggingface: https://huggingface.co/datasets/turkish_product_reviews

In [54]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer

In [55]:
dataset_name = 'turkish_product_reviews'
dataset = load_dataset(dataset_name)

Using custom data configuration default


Dataset turkish_product_reviews downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/turkish_product_reviews/default/1.0.0/d9158dc8210baf7a7ea495c26016996f6a27d7cc524ac239e5b8dcedcd292390. Subsequent calls will reuse this data.


We will only take 10% of the data to reduce training time

In [ ]:
sample = dataset['train'].train_test_split(test_size=0.1)

Now we split the data into training set (90%) and test set (10%)

In [ ]:
dataset = sample['test']
train_test = dataset.train_test_split(test_size=0.1)

In [ ]:
train_dataset = train_test['train']
test_dataset = train_test['test']

Now we can inspect the training data

In [ ]:
df_train = pd.DataFrame(train_dataset)

In [ ]:
df_train[['sentence', 'labels']].iloc[15:20]

Before we can start the training we need to tokenize the data save it in S3

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['sentence'], padding='max_length', truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

In [1]:
s3_prefix = 'datasets/turkish_product_reviews'

In [ ]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sagemaker_session_bucket}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sagemaker_session_bucket}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path,fs=s3)

### Model Training

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 8,
                 'model_name': model_name
                 }

In [ ]:
huggingface_estimator_sentiment = HuggingFace(entry_point='train.py',
                                    source_dir='./scripts',
                                    instance_type='ml.p3.2xlarge',
                                    instance_count=1,
                                    role=role,
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',
                                    hyperparameters = hyperparameters)

In [ ]:
huggingface_estimator_sentiment.fit({'train': training_input_path, 'test': test_input_path})

### Model Deployment

In [30]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model_sentiment = HuggingFaceModel(
   model_data= f"s3://{sagemaker_session_bucket}/huggingface-pytorch-training-2021-10-01-16-52-39-333/output/model.tar.gz",
   role=role,
   transformers_version="4.6", 
   pytorch_version="1.7", 
   py_version="py36", 
)

In [31]:
predictor_sentiment = huggingface_model_sentiment.deploy(1, "ml.m5.xlarge")

----------------!

### Model Testing

In [50]:
# Input text: "This is a pretty bad product, I wouldn't recommend this to anyone"
sentiment_input= {"inputs": "Bu oldukça kötü bir ürün, bunu kimseye tavsiye etmem"}
predictor_sentiment.predict(sentiment_input)

[{'label': 'LABEL_0', 'score': 0.9829983711242676}]

In [51]:
#Input text: "I love this shampoo, it makes my hair so shiny"
sentiment_input= {"inputs": "Bu şampuanı seviyorum, saçlarımı çok parlak yapıyor"}
predictor_sentiment.predict(sentiment_input)

[{'label': 'LABEL_1', 'score': 0.9976404905319214}]

## Question Answering

### Downloading the data

Taken from https://github.com/TQuad/turkish-nlp-qa-dataset

In [ ]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json data/train-v0.1.json

In [ ]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json data/dev-v0.1.json

The JSON files must be converted so that they can be used in a Q&A model

In [ ]:
import json
from datasets import load_dataset

def convert_json(input_filename, output_filename):
    with open(input_filename) as f:
        dataset = json.load(f)

    with open(output_filename, "w") as f:
        for article in dataset["data"]:
            title = article["title"]
            for paragraph in article["paragraphs"]:
                context = paragraph["context"]
                answers = {}
                for qa in paragraph["qas"]:
                    question = qa["question"]
                    idx = qa["id"]
                    answers["text"] = [a["text"] for a in qa["answers"]]
                    answers["answer_start"] = [int(a["answer_start"]) for a in qa["answers"]]
                    f.write(
                        json.dumps(
                            {
                                "id": idx,
                                "title": title,
                                "context": context,
                                "question": question,
                                "answers": answers,
                            }
                        )
                    )
                    f.write("\n")

In [ ]:
convert_json('data/train-v0.1.json', 'data/train.json')
convert_json('data/dev-v0.1.json', 'data/val.json')

In [22]:
data_files = {}
data_files["train"] = 'data/train.json'
data_files["validation"] = 'data/val.json'

In [57]:
from datasets import load_dataset
ds = load_dataset("json", data_files=data_files)

Using custom data configuration default-8dc8d5e84c5485b6
Reusing dataset json (/home/ec2-user/.cache/huggingface/datasets/json/default-8dc8d5e84c5485b6/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [58]:
df = pd.DataFrame(ds['train'])

In [60]:
df.iloc[7518:7521]

,id,title,context,question,answers
7518,8348,İstanbul Üniversitesi Gözlemevi,"İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi, 1933 yılında Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla açılmıştır. Gözlemsel astronomi konusunda çalışmalara evsahipliği yapan merkez;\r\n\r\nYıldız, güneş, uydu, astroid, kuyrukluyıldız, meteor, metorit ve tutulma gözlemleri yapmak ve gözlem verilerini değerlendirmek. \r\nDünyanın sayılı 200 gözlem veri merkezi ile 1939 yılından bu yana sürdürülen veri alışverişini sürdürmek. \r\nNASA, ESA gibi kuruluşların atmosfer dışından gözlem yapmak amacıyla uzaya gönderdiği yapay uydu verilerini alıp indirgemek ve değerlendirmek\r\nGerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak. gibi amaçlar gütmektedir.",İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi'nin kurulma amacı nedir?,"{'text': [' Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması '], 'answer_start': [75]}"
7519,8349,İstanbul Üniversitesi Gözlemevi,"İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi, 1933 yılında Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla açılmıştır. Gözlemsel astronomi konusunda çalışmalara evsahipliği yapan merkez;\r\n\r\nYıldız, güneş, uydu, astroid, kuyrukluyıldız, meteor, metorit ve tutulma gözlemleri yapmak ve gözlem verilerini değerlendirmek. \r\nDünyanın sayılı 200 gözlem veri merkezi ile 1939 yılından bu yana sürdürülen veri alışverişini sürdürmek. \r\nNASA, ESA gibi kuruluşların atmosfer dışından gözlem yapmak amacıyla uzaya gönderdiği yapay uydu verilerini alıp indirgemek ve değerlendirmek\r\nGerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak. gibi amaçlar gütmektedir.",İstanbul Üniversitesi Gözlemevinin ne gibi amaçları vardır?,"{'text': ['Gerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak'], 'answer_start': [661]}"
7520,8350,İstanbul Üniversitesi Gözlemevi,"İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi, 1933 yılında Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla açılmıştır. Gözlemsel astronomi konusunda çalışmalara evsahipliği yapan merkez;\r\n\r\nYıldız, güneş, uydu, astroid, kuyrukluyıldız, meteor, metorit ve tutulma gözlemleri yapmak ve gözlem verilerini değerlendirmek. \r\nDünyanın sayılı 200 gözlem veri merkezi ile 1939 yılından bu yana sürdürülen veri alışverişini sürdürmek. \r\nNASA, ESA gibi kuruluşların atmosfer dışından gözlem yapmak amacıyla uzaya gönderdiği yapay uydu verilerini alıp indirgemek ve değerlendirmek\r\nGerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak. gibi amaçlar gütmektedir.",İstanbul Üniversitesi Gözlemevi hangi amaçla açılmıştır?,"{'text': ['Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla'], 'answer_start': [76]}"


Uploading to S3

In [ ]:
s3_prefix = 'datasets/turkish_qa'

In [ ]:
!aws s3 cp data/train.json s3://$sagemaker_session_bucket/$s3_prefix/train.json
!aws s3 cp data/val.json s3://$sagemaker_session_bucket/$s3_prefix/val.json

### Model Training

In [ ]:
from sagemaker.huggingface import HuggingFace

hyperparameters={
    'model_name_or_path': model_name,
    'train_file': '/opt/ml/input/data/train/train.json',
    'validation_file': '/opt/ml/input/data/val/val.json',
    'do_train': True,
    'do_eval': False,
    'fp16': True,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'pad_to_max_length': True,
    'doc_stride': 128,
    'output_dir': '/opt/ml/model'
}

instance_type = 'ml.p3.16xlarge'
instance_count = 1
volume_size = 200

In [ ]:
huggingface_estimator = HuggingFace(entry_point='run_qa.py',
                                    source_dir='./scripts',
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    role=role,
                                    transformers_version='4.10',
                                    pytorch_version='1.9',
                                    py_version='py38',
                                    hyperparameters= hyperparameters,
                                    disable_profiler=True,
                                   )

In [ ]:
huggingface_estimator.fit({'train': f's3://{sagemaker_session_bucket}/{s3_prefix}/', 'val': f's3://{sagemaker_session_bucket}/{s3_prefix}/'})

### Model Deployment

In [36]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model_qa = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-10-12-05-07-28-087/output/model.tar.gz",
   role=role, # 
   transformers_version="4.10", 
   pytorch_version="1.9", 
   py_version="py38", 
)

In [37]:
predictor_qa = huggingface_model_qa.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

### Model Testing

In [52]:
#Question: "When did he start a vagabond life?"
#Predicted answer: "On his father's death"

data = {
"inputs": {
    "question": "Ne zaman avare bir hayata başladı?",
    "context": """ABASIYANIK, Sait Faik. Hikayeci (Adapazarı 23 Kasım 1906-İstanbul 11 Mayıs 1954). \
İlk öğrenimine Adapazarı’nda Rehber-i Terakki Mektebi’nde başladı. İki yıl kadar Adapazarı İdadisi’nde okudu.\
İstanbul Erkek Lisesi’nde devam ettiği orta öğrenimini Bursa Lisesi’nde tamamladı (1928). İstanbul Edebiyat \
Fakültesi’ne iki yıl devam ettikten sonra babasının isteği üzerine iktisat öğrenimi için İsviçre’ye gitti. \
Kısa süre sonra iktisat öğrenimini bırakarak Lozan’dan Grenoble’a geçti. Üç yıl başıboş bir edebiyat öğrenimi \
gördükten sonra babası tarafından geri çağrıldı (1933). Bir müddet Halıcıoğlu Ermeni Yetim Mektebi'nde Türkçe \
gurup dersleri öğretmenliği yaptı. Ticarete atıldıysa da tutunamadı. Bir ay Haber gazetesinde adliye muhabirliği\
yaptı (1942). Babasının ölümü üzerine aileden kalan emlakin geliri ile avare bir hayata başladı. Evlenemedi.\
Yazları Burgaz adasındaki köşklerinde, kışları Şişli’deki apartmanlarında annesi ile beraber geçen bu fazla \
içkili bohem hayatı ömrünün sonuna kadar sürdü."""
    }
}
predictor_qa.predict(data)['answer']

'Babasının ölümü üzerine'

In [53]:
#Question: "Why did Einstein leave Germany?"
#Predicted answer: "Due to the rise to power of the Nazi Party"

data = {
"inputs": {
    "question": "Einstein Almanya'dan neden ayrıldı?",
    "context": """Alman İmparatorluğu'nun Ulm kentinde, Aşkenazi Yahudi bir ailede dünyaya gelen Einstein, \ 
    yaşamının ilk yıllarını Münih'te geçirdi. Lise eğitimini ve yüksek eğitimini İsviçre'de tamamladı; fakat \
    bir üniversitede iş bulmada yaşadığı zorluklar nedeniyle bir patent ofisinde müfettiş olarak çalışmaya başladı. \
    1905 yılı Einstein için bir mucize yıl oldu ve o dönemde kuramları hemen benimsenmemiş olsa da ileride fizikte \
    devrim yaratacak olan dört makale yayımladı. 1914 yılında Max Planck'ın kişisel ricası ile Almanya'ya geri döndü. \
    1921 yılında fotoelektrik etki üzerine çalışmaları nedeniyle Nobel Fizik Ödülü'ne layık görüldü. Nazi Partisi'nin \
    iktidara yükselişi nedeniyle 1933'te Almanya'yı terk etti ve Amerika Birleşik Devletleri'ne yerleşti. Ömrünün geri \
    kalanını geçirdiği New Jersey eyaletinin Princeton ilçesinde ölmüştür. """
    }
}
predictor_qa.predict(data)['answer']

"Nazi Partisi'nin     iktidara yükselişi nedeniyle"